In [25]:
from torch import nn

In [26]:
def compute_total_number_of_flops(model, batch_size=1):
    total_flops = 0
    for name, layer in model.named_children():
        if isinstance(layer, nn.Linear):
            total_flops += layer.in_features * layer.out_features * batch_size
        elif isinstance(layer, nn.Conv1d):
            total_flops += layer.in_channels * layer.out_channels * layer.kernel_size[0] * batch_size
        elif isinstance(layer, nn.Conv2d):
            total_flops += 2 * layer.in_channels * layer.out_channels * layer.kernel_size[0] * batch_size

    return total_flops

In [27]:
class FemnistConfig(object):
    image_shape = (1, 28, 28)
    conv_channels = [32, 64]  # [20, 50]
    conv_kernels = [5, 5]  # [3, 3]
    conv_strides = [1, 1]
    lin_channels = [4*4*64, 2048]  # [5*5*50, 500]
    n_classes = 62

In [28]:
from torch import nn
import torch.nn.functional as t_func


class FemnistModel(nn.Module):
    def __init__(self, ):
        super(FemnistModel, self).__init__()
        self.config = FemnistConfig()
        # Convolution layers
        self.conv1 = nn.Conv2d(in_channels=self.config.image_shape[0],
                               out_channels=self.config.conv_channels[0],
                               kernel_size=self.config.conv_kernels[0],
                               stride=self.config.conv_strides[0])
        self.conv2 = nn.Conv2d(in_channels=self.config.conv_channels[0],
                               out_channels=self.config.conv_channels[1],
                               kernel_size=self.config.conv_kernels[1],
                               stride=self.config.conv_strides[1])
        # Fully connected layers
        self.fc1 = nn.Linear(in_features=self.config.lin_channels[0],
                             out_features=self.config.lin_channels[1])
        self.fc2 = nn.Linear(in_features=self.config.lin_channels[1],
                             out_features=self.config.n_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = t_func.relu(x)
        x = t_func.max_pool2d(x, 2, 2)
        x = self.conv2(x)
        x = t_func.relu(x)
        x = t_func.max_pool2d(x, 2, 2)
        x = x.view(-1, self.config.lin_channels[0])
        x = self.fc1(x)
        x = t_func.relu(x)
        return t_func.softmax(self.fc2(x), dim=1)

In [29]:
class SentConfig(object):
    embs_file = 'enea_fl/models/embs.json'
    cnn_num_channels = 100
    cnn_kernel_size = [3, 4, 5]
    lstm_layers = 2
    lstm_hidden = 20
    lstm_bidirectional = False
    output_size = 2
    max_sen_len = 30
    dropout = 0.3

In [30]:
import subprocess
import json
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as t_func


class CnnSent(nn.Module):
    def __init__(self, embs=None):
        super(CnnSent, self).__init__()
        self.config = SentConfig()
        if embs is None:
            try:
                print("Loading GloVe embeddings [{}]...".format(self.config.embs_file))
                with open(self.config.embs_file, 'r') as inf:
                    embs = json.load(inf)
            except FileNotFoundError:
                _ = subprocess.call("./enea_fl/models/get_embs.sh", shell=True)
                with open(self.config.embs_file, 'r') as inf:
                    embs = json.load(inf)
        self.embs = embs
        print("Loaded GloVe embeddings.")
        vocab = self.embs['vocab']
        vocab_size = len(vocab)
        print("Vocab size: {}".format(vocab_size))
        word_embeddings = torch.from_numpy(np.array(self.embs['emba'])).type(torch.FloatTensor)
        word_embeddings_size = word_embeddings.shape[1]
        # Embedding Layer
        self.embeddings = nn.Embedding(vocab_size, word_embeddings_size)
        self.embeddings.weight = nn.Parameter(word_embeddings, requires_grad=False)
        # Convolution Layers
        self.conv1 = nn.Conv1d(in_channels=word_embeddings_size, out_channels=self.config.cnn_num_channels,
                               kernel_size=self.config.cnn_kernel_size[0])
        self.kernel_size1 = self.config.max_sen_len - self.config.cnn_kernel_size[0] + 1
        self.conv2 = nn.Conv1d(in_channels=word_embeddings_size, out_channels=self.config.cnn_num_channels,
                               kernel_size=self.config.cnn_kernel_size[1])
        self.kernel_size2 = self.config.max_sen_len - self.config.cnn_kernel_size[1] + 1
        self.conv3 = nn.Conv1d(in_channels=word_embeddings_size, out_channels=self.config.cnn_num_channels,
                               kernel_size=self.config.cnn_kernel_size[2])
        self.kernel_size3 = self.config.max_sen_len - self.config.cnn_kernel_size[2] + 1
        self.dropout = nn.Dropout(self.config.dropout)
        # Fully connected layer
        self.fc = nn.Linear(self.config.cnn_num_channels * len(self.config.cnn_kernel_size), self.config.output_size)

    def forward(self, x):
        # Embed input to GloVe embeddings
        embedded_sent = self.embeddings(x)
        embedded_sent = embedded_sent.permute(1, 2, 0)
        # First convolution
        out1 = t_func.max_pool1d(t_func.relu(self.conv1(embedded_sent)), kernel_size=self.kernel_size1).squeeze(2)
        # Second convolution
        out2 = t_func.max_pool1d(t_func.relu(self.conv2(embedded_sent)), kernel_size=self.kernel_size3).squeeze(2)
        # Third convolution
        out3 = t_func.max_pool1d(t_func.relu(self.conv3(embedded_sent)), kernel_size=self.kernel_size3).squeeze(2)
        # Aggregate convolutions
        all_out = torch.cat((out1, out2, out3), 1)
        # Fully connected and output
        final_feature_map = self.dropout(all_out)
        return t_func.softmax(self.fc(final_feature_map), dim=1)

In [31]:
class Nbaiot(nn.Module):
    def __init__(self, input_shape=31, nb_classes=11):
        super(Nbaiot, self).__init__()
        self.fc1 = nn.Linear(input_shape, 64)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(64, 32)
        self.relu2 = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        self.fc3 = nn.Linear(32, nb_classes)
        # self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.dropout(x)
        x = self.fc3(x)
        # x = self.softmax(x)
        return x
    

In [32]:
compute_total_number_of_flops(FemnistModel(), batch_size=1)

2244928

In [33]:
compute_total_number_of_flops(CnnSent(), batch_size=1)

Loading GloVe embeddings [enea_fl/models/embs.json]...
Loaded GloVe embeddings.
Vocab size: 400000


60600

In [34]:
from thop import profile
from thop import clever_format

model = FemnistModel()
input = torch.randn(1, 1, 28, 28)
macs, params = profile(model, inputs=(input, ))
macs, params = clever_format([macs, params], "%.3f")
total_params = sum(p.numel() for p in model.parameters())
print("Total params: {}".format(total_params))
print("Total MACs: {}".format(macs))
print("Total FLOPs: {}".format(params))


[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
Total params: 2278334
Total MACs: 5.962M
Total FLOPs: 2.278M


In [42]:
model = CnnSent()
input = torch.randint(0, 100, (30 ,10))
macs, params = profile(model, inputs=(input, ))
macs, params = clever_format([macs, params], "%.3f")
total_params = sum(p.numel() for p in model.parameters())
print("Total params: {}".format(total_params))
print("Total MACs: {}".format(macs))
print("Total FLOPs: {}".format(params))

Loading GloVe embeddings [enea_fl/models/embs.json]...
Loaded GloVe embeddings.
Vocab size: 400000
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv1d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
Total params: 20060952
Total MACs: 16.106M
Total FLOPs: 60.902K


In [43]:
model = Nbaiot()
input = torch.randn(1,1,31)
macs, params = profile(model, inputs=(input, ))
macs, params = clever_format([macs, params], "%.3f")
total_params = sum(p.numel() for p in model.parameters())
print("Total params: {}".format(total_params))
print("Total MACs: {}".format(macs))
print("Total FLOPs: {}".format(params))

[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
Total params: 4491
Total MACs: 4.384K
Total FLOPs: 4.491K


In [ ]:
total_params = sum(p.numel() for p in model.parameters())